<a href="https://colab.research.google.com/github/siyuduan6/safedrivechicago/blob/master/partial_face_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression     # 线性模型中的 Logistic 回归模型
from sklearn.neural_network import MLPClassifier        # 神经网络模型中的多层网络模型
from sklearn.svm import LinearSVC                       # SVM 模型中的线性 SVC 模型
from sklearn.linear_model import SGDClassifier          # 线性模型中的随机梯度下降模型
from sklearn.linear_model import Ridge              #线性模型中的Ridge模型
from sklearn.multioutput import MultiOutputRegressor

In [3]:
# ds = pd.read_csv("https://raw.githubusercontent.com/arxrean/PartialFace/main/repo/celeba.csv?token=ARVBZLPD3PTRNCDNIWRUHD3BFF73W")

In [4]:
# ds

,id,identity,path,lm68,identity_idx,left_eye,right_eye,nose,mouth
0,000024.jpg,9290,./celeba/0/000024.jpg,"[[48, 118], [48, 128], [49, 139], [51, 149], [...",0,"[48, 93, 85, 120]","[85, 94, 134, 133]","[78, 107, 99, 131]","[71, 131, 104, 179]"
1,000041.jpg,1058,./celeba/0/000041.jpg,"[[66, 111], [63, 123], [61, 135], [62, 147], [...",1,"[65, 90, 66, 142]","[65, 97, 166, 142]","[71, 109, 95, 152]","[66, 152, 105, 202]"
2,000043.jpg,5239,./celeba/0/000043.jpg,"[[41, 112], [43, 125], [45, 137], [47, 150], [...",2,"[41, 97, 89, 136]","[89, 98, 133, 136]","[77, 111, 100, 143]","[67, 143, 110, 189]"
3,000050.jpg,1058,./celeba/0/000050.jpg,"[[38, 114], [39, 127], [40, 140], [42, 153], [...",1,"[38, 99, 93, 138]","[93, 99, 129, 138]","[77, 112, 102, 145]","[67, 145, 111, 194]"
4,000056.jpg,545,./celeba/0/000056.jpg,"[[42, 115], [43, 126], [44, 138], [46, 148], [...",3,"[42, 98, 97, 132]","[97, 96, 122, 132]","[78, 111, 100, 140]","[75, 140, 103, 178]"
...,...,...,...,...,...,...,...,...,...
19038,202567.jpg,6246,./celeba/999/202567.jpg,"[[45, 109], [45, 121], [46, 133], [47, 145], [...",968,"[45, 101, 90, 129]","[90, 102, 121, 129]","[78, 109, 97, 138]","[78, 138, 98, 184]"
19039,202569.jpg,7859,./celeba/999/202569.jpg,"[[54, 109], [52, 119], [51, 130], [54, 141], [...",946,"[54, 95, 75, 138]","[75, 99, 149, 138]","[75, 112, 99, 145]","[68, 145, 112, 193]"
19040,202574.jpg,8234,./celeba/999/202574.jpg,"[[39, 120], [40, 131], [41, 141], [44, 152], [...",906,"[39, 96, 93, 127]","[93, 96, 123, 129]","[76, 110, 100, 136]","[74, 136, 104, 176]"
19041,202589.jpg,7982,./celeba/999/202589.jpg,"[[51, 115], [52, 126], [52, 137], [53, 148], [...",953,"[51, 101, 82, 130]","[82, 98, 139, 130]","[77, 111, 101, 139]","[69, 139, 106, 188]"


In [181]:
def xy_set():
  ds = pd.read_csv("https://raw.githubusercontent.com/arxrean/PartialFace/main/repo/celeba.csv?token=ARVBZLPD3PTRNCDNIWRUHD3BFF73W")# The link of celeba.csv file on github, which should be updated
  ds1 = ds[['id', 'identity', 'path', 'lm68', 'identity_idx', 'left_eye']].rename(columns = {"left_eye" : "partial"})
  ds2 = ds[['id', 'identity', 'path', 'lm68', 'identity_idx', 'right_eye']].rename(columns = {"right_eye" : "partial"})
  ds3 = ds[['id', 'identity', 'path', 'lm68', 'identity_idx', 'nose']].rename(columns = {"nose" : "partial"})
  ds4 = ds[['id', 'identity', 'path', 'lm68', 'identity_idx', 'mouth']].rename(columns = {"mouth" : "partial"})
  ds1["index"] = 0
  ds2["index"] = 1
  ds3["index"] = 2
  ds4["index"] = 3
  xy_dataset = pd.concat([ds1,ds2,ds3,ds4]).reset_index().drop(columns="level_0")
  return xy_dataset



In [ ]:
def dataset_for_modeling(baseline_dataset):
  X_set = baseline_dataset[["partial","index"]]
  Y_set = baseline_dataset["lm68"]
  x_set = pd.concat([X_set["index"],X_set['partial'].str.replace("[","").str.replace("]","").str.split(',', expand=True)],axis=1)
  y_set = pd.concat([Y_set.str.replace("[","").str.replace("]","").str.split(',', expand=True)], axis=1)
  return x_set, y_set


In [28]:
baseline_dataset = pd.read_csv("dataset_for_modeling.csv")

# Model 

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)

In [ ]:
def model_Ridge():
  x_set, y_set = dataset_for_modeling(xy_set())
  X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)
  model_ridge = MultiOutputRegressor(Ridge(random_state=123)).fit(X_train, y_train)
  score = model_ridge.score(X_test, y_test)
  print(model_ridge, score)
  return model_ridge


In [ ]:
def model_LR():
  x_set, y_set = dataset_for_modeling(xy_set())
  X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)
  model_lr = MultiOutputRegressor(LogisticRegression(random_state = 0)).fit(X_train, y_train)
  score = model_lr.score(X_test, y_test)
  print(model_lr, score)
  return model_lr

In [ ]:
def model_svm():
  x_set, y_set = dataset_for_modeling(xy_set())
  X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)
  model_svm = MultiOutputRegressor(LinearSVC(random_state=0, tol=1e-5)).fit(X_train, y_train)
  score = model_svm.score(X_test, y_test)
  print(model_svm, score)
  return model_svm

In [ ]:
def model_SGD():
  x_set, y_set = dataset_for_modeling(xy_set())
  X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)
  model_sgd = MultiOutputRegressor(SGDClassifier(max_iter=500, tol=1e-3)).fit(X_train, y_train)
  score = model_sgd.score(X_test, y_test)
  print(model_sgd, score)
  return model_sgd


In [ ]:
def model_NN():
  x_set, y_set = dataset_for_modeling(xy_set())
  X_train, X_test, y_train, y_test = train_test_split(x_set,y_set,test_size=0.3,random_state = 42)
  model_nn = MultiOutputRegressor(MLPClassifier(random_state=1, max_iter=300)).fit(X_train, y_train)
  score = model_nn.score(X_test, y_test)
  print(model_nn, score)
  return model_nn


In [ ]:
def predict():
  index = input("Index:")
  left = input("Left:")
  top = input("Top:")
  right = input("Right:")
  bottom = input("Bottom:")
  input = [[index,left,top,right,bottom]]
  for model in ()
  output = model_Ridge().predict(input)
  lm68 = np.narray(shape=(2,68),dtype = int)
  print("lm68:"+ lm68)
  return lm68


In [49]:
# mor.predict([[3,73,145,105,177]])

array([[ 47.27251783, 112.68129251,  47.84646888, 123.76235031,
         49.3769744 , 134.69460828,  51.40893696, 145.47560205,
         55.15783703, 155.7261085 ,  61.32609874, 164.73581782,
         69.21891897, 172.38300676,  78.27738314, 178.68440066,
         88.66481194, 180.44261223,  99.15538564, 178.7564937 ,
        108.56093914, 172.67988869, 116.87715455, 165.20590829,
        123.41322106, 156.23326691, 127.34830022, 145.91645707,
        129.44523918, 134.93276545, 130.97922276, 123.80911249,
        131.64514375, 112.50902742,  53.62484785, 106.22206344,
         59.29685727, 102.33583925,  66.61246851, 101.91834927,
         73.94241618, 103.32238133,  81.01590666, 106.10696137,
         95.59344646, 105.9647135 , 102.87819488, 103.15298425,
        110.40298314, 101.74459184, 117.92894919, 102.30557606,
        123.85839429, 106.14511213,  88.34934372, 113.03061413,
         88.28703874, 120.30621287,  88.21303436, 127.70432428,
         88.14902601, 135.19417814,  80.